# Interfaz Gráfica para Predicciones

In [1]:
%%capture
!pip install gradio

In [13]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

### Creamos el esqueleto del modelo para cargarlo

In [6]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    # LSTM
    self.lstm = nn.LSTM(
        input_size,
        hidden_size,
        num_layers,
        dropout=dropout,
        batch_first=True
    )

    # Layer Normalization
    self.layer_norm = nn.LayerNorm(hidden_size)

    self.classifier = nn.Sequential(
        nn.Linear(hidden_size, hidden_size // 2),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_size // 2, hidden_size//4),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_size//4, output_size)
    )

  def forward(self, x):
    if len(x.shape) == 2:
        x = x.unsqueeze(1)  # Agrega dimensión de secuencia (batch_size, 1, embedding_dim)

    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

    # LSTM output: output of all the hidden_state, hidde_state, cell_state
    lstm_out, (hidden, _) = self.lstm(x, (h0, c0))

    # Apply Layer Normalization to the last output of the LSTM
    lstm_out_norm = self.layer_norm(lstm_out[:, -1, :])

    # Classification layers
    output = self.classifier(lstm_out_norm)
    return output

#### Función para cargar el modelo guardado

In [11]:
def cargar_modelo():
    embedding_dim = 768  # Embedding size
    hidden_dim = 256
    output_dim = 6  # Number of emotion or classes
    num_layers = 2
    dropout = 0.3

    # Model
    model = LSTM(embedding_dim, hidden_dim, num_layers, output_dim, dropout)
    model.load_state_dict(torch.load('model/modelo_emociones.pth', map_location=torch.device('cpu')))
    model.eval()
    return model

## Preprocesamiento y vectorizacion del Texto

In [8]:
def procesar_texto(text):
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
    model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

    with torch.no_grad():
        inputs = tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

## Predecir la emocion

In [9]:
def predict_sentiment(text):
    # Obtener los embeddings
    embeddings = procesar_texto(text)
    # Carga el modelo
    model = cargar_modelo()
    # Predicción
    with torch.no_grad():
        output = model(embeddings)
    probabilities = F.softmax(output, dim=1).squeeze().tolist()

    label_mapping = {
        0: "sadness",
        1: "joy",
        2: "love",
        3: "anger",
        4: "fear",
        5: "surprise"
    }

    # Obtener la clase con la mayor probabilidad
    predicted_index = torch.argmax(output, dim=1).item()

    # Genera un diccionario con las etiquetas y sus probabilidades
    probabilities_dict = {label_mapping[i]: prob for i, prob in enumerate(probabilities)}

    return probabilities_dict

## Interfaz Grafica

In [14]:
import gradio as gr

# Define la interfaz
interface = gr.Interface(
    fn=predict_sentiment,  # Función de predicción
    inputs=gr.Textbox(lines=3, placeholder="Escribe tu texto aquí..."),
    outputs=gr.Label(num_top_classes=6),  # Muestra todas las clases con probabilidades
    title="Análisis de Sentimientos",
    description="Ingresa un texto para analizar su sentimiento y ver la probabilidad de cada clase."
)

# Lanza la aplicación
interface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
